# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,portland,45.5234,-122.6762,17.97,56,100,4.47,US,1712115758
1,1,usinsk,65.9939,57.5281,-18.78,88,9,2.56,RU,1712116270
2,2,liwale,-9.7667,37.9333,20.66,98,82,1.37,TZ,1712116271
3,3,blackmans bay,-43.0167,147.3167,14.85,53,100,1.34,AU,1712116272
4,4,aasiaat,68.7098,-52.8699,-11.37,88,98,2.31,GL,1712116273


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points (
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    scale = 0.8,
    color = "City",
    alpha = 0.5,
    frame_width = 800,
    frame_height = 500
)   

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df = city_data_df.loc[(city_data_df['Max Temp'] > 21) & (city_data_df['Max Temp'] < 27) &
                            (city_data_df['Wind Speed'] < 4.5) & (city_data_df['Cloudiness'] == 0)]

# Drop any rows with null values
city_data_df = city_data_df.dropna (how = 'any', axis = 0)

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
22,22,pisco,-13.7000,-76.2167,22.03,78,0,0.00,PE,1712116294
48,48,arraial do cabo,-22.9661,-42.0278,23.92,88,0,1.03,BR,1712116322
121,121,margaret river,-33.9500,115.0667,22.88,37,0,0.83,AU,1712116403
178,178,salalah,17.0151,54.0924,26.05,47,0,2.06,OM,1712116465
234,234,puerto lempira,15.2667,-83.7667,23.96,83,0,4.39,HN,1712116527
266,266,jeddah,21.5169,39.2192,25.92,65,0,2.06,SA,1712116561
408,408,vila velha,-20.3297,-40.2925,25.97,92,0,0.51,BR,1712116571
418,418,sensuntepeque,13.8667,-88.6333,21.24,78,0,0.50,SV,1712116726
422,422,callao,-12.0667,-77.1500,24.84,73,0,3.60,PE,1712116449
434,434,ghat,24.9647,10.1728,22.61,13,0,2.21,LY,1712116745


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
22,pisco,PE,-13.7000,-76.2167,78,
48,arraial do cabo,BR,-22.9661,-42.0278,88,
121,margaret river,AU,-33.9500,115.0667,37,
178,salalah,OM,17.0151,54.0924,47,
234,puerto lempira,HN,15.2667,-83.7667,83,
266,jeddah,SA,21.5169,39.2192,65,
408,vila velha,BR,-20.3297,-40.2925,92,
418,sensuntepeque,SV,13.8667,-88.6333,78,
422,callao,PE,-12.0667,-77.1500,73,
434,ghat,LY,24.9647,10.1728,13,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"
limit = 1
params = {
    "categories":categories,
    "limit":limit,
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row['Lng']
    latitude = row['Lat']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
   
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
pisco - nearest hotel: La Portada
arraial do cabo - nearest hotel: No hotel found
margaret river - nearest hotel: Margaret River Hotel
salalah - nearest hotel: Muscat International Hotel
puerto lempira - nearest hotel: Hotel Yu Baiwan
jeddah - nearest hotel: No hotel found
vila velha - nearest hotel: Hotel Prainha
sensuntepeque - nearest hotel: Hotel Arizona
callao - nearest hotel: Casa Ronald
ghat - nearest hotel: فندق تاسيلي
dahab - nearest hotel: Lighthouse Hotel
ramon castilla - nearest hotel: Hotel Solimões


,City,Country,Lat,Lng,Humidity,Hotel Name
22,pisco,PE,-13.7000,-76.2167,78,La Portada
48,arraial do cabo,BR,-22.9661,-42.0278,88,No hotel found
121,margaret river,AU,-33.9500,115.0667,37,Margaret River Hotel
178,salalah,OM,17.0151,54.0924,47,Muscat International Hotel
234,puerto lempira,HN,15.2667,-83.7667,83,Hotel Yu Baiwan
266,jeddah,SA,21.5169,39.2192,65,No hotel found
408,vila velha,BR,-20.3297,-40.2925,92,Hotel Prainha
418,sensuntepeque,SV,13.8667,-88.6333,78,Hotel Arizona
422,callao,PE,-12.0667,-77.1500,73,Casa Ronald
434,ghat,LY,24.9647,10.1728,13,فندق تاسيلي


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_map_plot = hotel_df.hvplot.points (
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.7,
    hover_cols = ['Lng', 'Lat', 'City', 'Humidity', 'Hotel Name', 'Country'],
    frame_width = 800,
    frame_height = 500
)   

# Display the map
hotel_map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)